In [1]:
import os
import sys
# fix imports
# sys.path.append(os.path.dirname(os.getcwd()))
os.chdir(os.path.dirname(os.getcwd()))

from misc import dossier

from datetime import datetime 
import pandas as pd

In [2]:
# read custom data
data = pd.read_parquet("data/demo/custom_wvs7_data.parquet")

In [3]:
# shuffle data
sdata = data.sample(frac=1).reset_index(drop=True)

In [4]:
# generate dump_dates
date_range = list(pd.date_range(start="2024-01-01", end="2024-12-31", freq="MS"))

In [5]:
# write batches
lower_bound = 0
# define the slices (12 months)
upper_bound = threshold = len(sdata) / 12

for idx, vals in enumerate(date_range):
    # assign dates by slices
    sdata.loc[lower_bound:upper_bound, "dump_date"] = vals
    # udpate index ranges
    lower_bound += threshold
    upper_bound += threshold or None

In [6]:
# output batches
for date in list(sdata.dump_date.unique()):
    data_path = f"./data/demo/batches/wvs7_{date.date()}.parquet"
    sdata[sdata.dump_date == date].drop(columns="dump_date").to_parquet(data_path, index=False)